<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Performs Research with Multi-Agent Group Chat

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [2]:
%%capture --no-stderr
# %pip install pyautogen~=0.2.0b4

In [3]:
%pip install nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("vader_lexicon")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/vscode/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [4]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
         "model": ["gpt-3.5-turbo-16k-0613"],
        # "model": ["gpt-3.5-turbo"],
    },
)

# Import Settings

In [5]:
import json

gpt_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 1.0,  # Controls randomness of responses
    "config_list": config_list_gpt4,  # Presumably a list of additional GPT-3.5 configurations
    "timeout": 120,  # Maximum time for the agent to respond
}

# Load agent configurations from the JSON file
with open('agents_config_v2.json', 'r') as file:
    agents_config = json.load(file)


## Construct Agents

In [6]:
# Initialize agents using the configurations from the JSON file
Agents_1 = []
Agents_2 = []
Agents_3 = []

for key, config in agents_config.items():
    # Extracting relevant details from the new JSON structure
    role = config['Role']
    demographic = config['Demographic']
    opinions = config['Opinions']
    task = config['Task']
    format_requirement = config['Format']

    # Constructing a system message based on the new JSON structure
    system_message_1 = f"[Role] {role}\n[Task] {task}\n[Format] {format_requirement}"
    system_message_2 = f"[Role] {role}\n[Opinions] {opinions}\n[Task] {task}\n[Format] {format_requirement}"
    system_message_3 = f"[Role] {role}\n[Demographic] {demographic}\n[Opinions] {opinions}\n[Task] {task}\n[Format] {format_requirement}"

    # Creating the agent with the new system message
    agent_1 = autogen.AssistantAgent(
        name=key,  # 'key' is the name of the agent
        llm_config=gpt_config,
        system_message=system_message_1
    )
    agent_2 = autogen.AssistantAgent(
    name=key,  # 'key' is the name of the agent
    llm_config=gpt_config,
    system_message=system_message_1
)
    agent_3 = autogen.AssistantAgent(
    name=key,  # 'key' is the name of the agent
    llm_config=gpt_config,
    system_message=system_message_1
)
    Agents_1.append(agent_1)
    Agents_2.append(agent_2)
    Agents_3.append(agent_3)


user_proxy = autogen.UserProxyAgent(
   name="Government",
   system_message="A human admin. Interact with all other agents to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)

Agents_1.append(user_proxy)
Agents_2.append(user_proxy)
Agents_3.append(user_proxy)
print("Added user proxy")

# Create a group chat with all agents
groupchat_1 = autogen.GroupChat(
    agents = Agents_1,
    messages=[],
    max_round=100
)
groupchat_2 = autogen.GroupChat(
    agents = Agents_2,
    messages=[],
    max_round=100
)
groupchat_3 = autogen.GroupChat(
    agents = Agents_3,
    messages=[],
    max_round=100
)

manager_1 = autogen.GroupChatManager(groupchat=groupchat_1, llm_config=gpt_config)
manager_2 = autogen.GroupChatManager(groupchat=groupchat_2, llm_config=gpt_config)
manager_3 = autogen.GroupChatManager(groupchat=groupchat_3, llm_config=gpt_config)


Added user proxy


## Start Chat

In [9]:
# Round_1
user_proxy.initiate_chat(
    manager_1,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pros of Low-income housing]
Affordable Housing: Offers affordable living in otherwise expensive areas, diversifying and benefiting the community.
Economic Diversity: Enhances local economic diversity, contributing to a more balanced economy.
Community Services: Brings additional services and amenities, advantageous to all residents.
Urban Revitalization: Transforms underused areas into vibrant residential spaces, revitalizing urban landscapes.
Reduced Commute: Lowers commute time and costs for low-income individuals working nearby.
[Cons of Low-income housing]
Safety Concerns: May lead to increased crime and homelessness, affecting community security and social dynamics.
Financial Implications: Relies on government subsidies, raising concerns about long-term sustainability and taxpayer burden.
Community Displacement: Could displace current residents or businesses, altering neighborhood culture and social fabric.
Land Value Fluctuations: Might decrease surrounding land values, impacting homeowners' investments and local tax base.
Strain on Public Services: Risks overcrowding local schools and services, potentially decreasing quality and increasing maintenance costs.
[Pros of Shopping Mall]
Economic Boost: Generates jobs and stimulates local economy through retail and service opportunities.
Tax Revenue: Increases local tax revenues through business and property taxes.
Consumer Convenience: Provides residents with easy access to a variety of shops and services.
Increased Foot Traffic: Attracts visitors, benefiting nearby businesses and promoting overall economic growth.
Community Hub: Can serve as a social and cultural gathering place for the community.
[Cons of Shopping Mall]
Traffic Congestion: Could increase traffic and parking issues in the area.
Local Business Impact: May negatively affect small, local businesses unable to compete with larger retailers.
Environmental Concerns: Construction and operation could lead to increased pollution and resource consumption.
Economic Inequality: Tends to benefit higher-income groups, potentially widening economic disparities.
Cultural Homogenization: Risk of diluting local culture with generic, globalized retail spaces.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pr

In [7]:
# Round_2
user_proxy.initiate_chat(
    manager_2,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pros of Low-income housing]
Affordable Housing: Offers affordable living in otherwise expensive areas, diversifying and benefiting the community.
Economic Diversity: Enhances local economic diversity, contributing to a more balanced economy.
Community Services: Brings additional services and amenities, advantageous to all residents.
Urban Revitalization: Transforms underused areas into vibrant residential spaces, revitalizing urban landscapes.
Reduced Commute: Lowers commute time and costs for low-income individuals working nearby.
[Cons of Low-income housing]
Safety Concerns: May lead to increased crime and homelessness, affecting community security and social dynamics.
Financial Implications: Relies on government subsidies, raising concerns about long-term sustainability and taxpayer burden.
Community Displacement: Could displace current residents or businesses, altering neighborhood culture and social fabric.
Land Value Fluctuations: Might decrease surrounding land values, impacting homeowners' investments and local tax base.
Strain on Public Services: Risks overcrowding local schools and services, potentially decreasing quality and increasing maintenance costs.
[Pros of Shopping Mall]
Economic Boost: Generates jobs and stimulates local economy through retail and service opportunities.
Tax Revenue: Increases local tax revenues through business and property taxes.
Consumer Convenience: Provides residents with easy access to a variety of shops and services.
Increased Foot Traffic: Attracts visitors, benefiting nearby businesses and promoting overall economic growth.
Community Hub: Can serve as a social and cultural gathering place for the community.
[Cons of Shopping Mall]
Traffic Congestion: Could increase traffic and parking issues in the area.
Local Business Impact: May negatively affect small, local businesses unable to compete with larger retailers.
Environmental Concerns: Construction and operation could lead to increased pollution and resource consumption.
Economic Inequality: Tends to benefit higher-income groups, potentially widening economic disparities.
Cultural Homogenization: Risk of diluting local culture with generic, globalized retail spaces.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pr

In [6]:
# Round_3
user_proxy.initiate_chat(
    manager_3,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pros of Low-income housing]
Affordable Housing: Offers affordable living in otherwise expensive areas, diversifying and benefiting the community.
Economic Diversity: Enhances local economic diversity, contributing to a more balanced economy.
Community Services: Brings additional services and amenities, advantageous to all residents.
Urban Revitalization: Transforms underused areas into vibrant residential spaces, revitalizing urban landscapes.
Reduced Commute: Lowers commute time and costs for low-income individuals working nearby.
[Cons of Low-income housing]
Safety Concerns: May lead to increased crime and homelessness, affecting community security and social dynamics.
Financial Implications: Relies on government subsidies, raising concerns about long-term sustainability and taxpayer burden.
Community Displacement: Could displace current residents or businesses, altering neighborhood culture and social fabric.
Land Value Fluctuations: Might decrease surrounding land values, impacting homeowners' investments and local tax base.
Strain on Public Services: Risks overcrowding local schools and services, potentially decreasing quality and increasing maintenance costs.
[Pros of Shopping Mall]
Economic Boost: Generates jobs and stimulates local economy through retail and service opportunities.
Tax Revenue: Increases local tax revenues through business and property taxes.
Consumer Convenience: Provides residents with easy access to a variety of shops and services.
Increased Foot Traffic: Attracts visitors, benefiting nearby businesses and promoting overall economic growth.
Community Hub: Can serve as a social and cultural gathering place for the community.
[Cons of Shopping Mall]
Traffic Congestion: Could increase traffic and parking issues in the area.
Local Business Impact: May negatively affect small, local businesses unable to compete with larger retailers.
Environmental Concerns: Construction and operation could lead to increased pollution and resource consumption.
Economic Inequality: Tends to benefit higher-income groups, potentially widening economic disparities.
Cultural Homogenization: Risk of diluting local culture with generic, globalized retail spaces.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pr

KeyboardInterrupt: Interrupted by user

In [7]:
# Round_3_2
user_proxy.initiate_chat(
    manager_3,
    message="""
[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pros of Low-income housing]
Affordable Housing: Offers affordable living in otherwise expensive areas, diversifying and benefiting the community.
Economic Diversity: Enhances local economic diversity, contributing to a more balanced economy.
Community Services: Brings additional services and amenities, advantageous to all residents.
Urban Revitalization: Transforms underused areas into vibrant residential spaces, revitalizing urban landscapes.
Reduced Commute: Lowers commute time and costs for low-income individuals working nearby.
[Cons of Low-income housing]
Safety Concerns: May lead to increased crime and homelessness, affecting community security and social dynamics.
Financial Implications: Relies on government subsidies, raising concerns about long-term sustainability and taxpayer burden.
Community Displacement: Could displace current residents or businesses, altering neighborhood culture and social fabric.
Land Value Fluctuations: Might decrease surrounding land values, impacting homeowners' investments and local tax base.
Strain on Public Services: Risks overcrowding local schools and services, potentially decreasing quality and increasing maintenance costs.
[Pros of Shopping Mall]
Economic Boost: Generates jobs and stimulates local economy through retail and service opportunities.
Tax Revenue: Increases local tax revenues through business and property taxes.
Consumer Convenience: Provides residents with easy access to a variety of shops and services.
Increased Foot Traffic: Attracts visitors, benefiting nearby businesses and promoting overall economic growth.
Community Hub: Can serve as a social and cultural gathering place for the community.
[Cons of Shopping Mall]
Traffic Congestion: Could increase traffic and parking issues in the area.
Local Business Impact: May negatively affect small, local businesses unable to compete with larger retailers.
Environmental Concerns: Construction and operation could lead to increased pollution and resource consumption.
Economic Inequality: Tends to benefit higher-income groups, potentially widening economic disparities.
Cultural Homogenization: Risk of diluting local culture with generic, globalized retail spaces.
""",
)

Government (to chat_manager):


[Role] I am the government for the Kendall Square renovation project.
[Background] An old government building in the center of Kendall Square will be torn down for renovation. Our proposed plan involves transforming an unused government building into affordable residential units, specifically designed to cater to low-income individuals and families, thereby addressing the issue of lack of low-income housing in the area. 
[Financial Limits] We have a tight total budget of 25 million tax money from all the stakeholders.
[Projects] We can only choose one of the two options
Low-income Housing: Construction of housing units: $150,000 - $250,000 per unit. Total for 100 units: $15 million - $25 million. It mainly funded by tax money and non-profit.
Shopping Mall: Construction of a medium-sized shopping mall: $250 - $500 per square foot. For a 100,000 sq ft mall: $25 million - $50 million. The shopping mall doesn't require tax money and will generate profit.
[Pr